In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 0. Module installation
### ref to https://www.kaggle.com/code/leonidkulyk/lb-0-45836-blip-clip-clip-interrogator

In [ ]:
#!pip install --no-index /kaggle/input/transformer-4180/transformers_4.18.0/sacremoses-0.0.53-py3-none-any.whl
#!pip install --no-index -q /kaggle/input/transformer-4180/transformers_4.18.0/transformers-4.18.0.dev0-py3-none-any.whl
# Using the pre compiled wheel since we don't have internet on submission
!pip install -q /kaggle/input/stable-diffusion-data/transformers-4.18.0.dev0-py3-none-any.whl

from transformers import OFATokenizer, OFAModel
from transformers.models.ofa.generate import sequence_generator

In [ ]:
import os
import sys
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt 
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate

from torchvision import transforms
from sklearn.preprocessing import normalize

import numpy as np
import pandas as pd
import torch

!pip install --no-index --find-links /kaggle/input/clip-interrogator-wheels-x /kaggle/input/clip-interrogator-wheels-x/clip_interrogator-0.4.3-py3-none-any.whl -q
import inspect
import importlib

from blip.models import blip
import open_clip
from clip_interrogator import clip_interrogator

sys.path.append('/kaggle/input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import SentenceTransformer, models, util

In [ ]:
# fix clip_interrogator bug
clip_interrogator_path = inspect.getfile(clip_interrogator.Interrogator)

fin = open(clip_interrogator_path, "rt")
data = fin.read()
data = data.replace(
    'open_clip.get_tokenizer(clip_model_name)', 
    'open_clip.get_tokenizer(config.clip_model_name.split("/", 2)[0])'
)
fin.close()

fin = open(clip_interrogator_path, "wt")
fin.write(data)
fin.close()

# reload module
importlib.reload(clip_interrogator)

# 1. Build models
### standard model
### Chose all-MiniLM-L6-v2
### This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

## 1.0 Preparation

In [ ]:
#model = SentenceTransformer('all-MiniLM-L6-v2')
st_model = SentenceTransformer("/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2")

### Prepare functions of encode, scoring, plotting

In [ ]:
import matplotlib.image as mpimg
import textwrap

def Prompt2Emb(input_li):
    if isinstance(input_li,str):
        input_li = [input_li]
    out_li = []
    for element in input_li:
        if type(element) is str:
            out = st_model.encode(element, show_progress_bar=False)
        else:
            out = element
        if isinstance(out, torch.Tensor):
            out = out.detach().cpu().numpy()
        out_li.append(out)
    output_li = out_li[0] if isinstance(input_li,str) else out_li
    return output_li

def scoring(prompt_li, prompt_clip_li):
    score_li=[]
    prompt_emb_li = Prompt2Emb(prompt_li)
    pre_prompt_emb_li = Prompt2Emb(prompt_clip_li)
    for prompt_emb, predict_emb in zip(prompt_emb_li, pre_prompt_emb_li):
        cosine_similarity_score = util.cos_sim(prompt_emb.astype(float), predict_emb.astype(float)).numpy().flatten()
        score_li.append(cosine_similarity_score[0])
    return score_li

def image_grid(imgs, prompts='', prompts2='', score_li='', rows=1, cols=7):
    assert len(imgs) == rows*cols
    Figure=plt.figure(figsize=(18,15))
    for idx,img in enumerate(imgs):
        ax=Figure.add_subplot(cols,rows,idx+1)
        ax.imshow(img)
        ax.tick_params(bottom=False,left=False,labelbottom=False,labelleft=False)
        if score_li!='':
            ax.annotate('score: %.3f'%(score_li[idx]),xycoords='axes fraction',ha='left',va='top',xy=(1.1,1.0),wrap=True,fontsize=12,c='r')
        if prompts!='':
            prompt = prompts[idx]
            text= '\n'.join(textwrap.wrap(prompt,60))
            ax.annotate(text,xycoords='axes fraction',ha='left',va='top',xy=(1.1,0.9),wrap=True,fontsize=12,c='k')
        if prompts2!='':
            prompt2 = prompts2[idx]
            text2= '\n'.join(textwrap.wrap(prompt2,60))
            ax.annotate(text2,xycoords='axes fraction',ha='left',va='top',xy=(1.1,0.3),wrap=True,fontsize=12,c='b')
    plt.show()

In [ ]:
submission = True

## 1.1 CLIP interrogator model

### 1.1.1 build model

In [ ]:
class CFG:
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
#    seed = 42
    embedding_length = 384
#    sentence_model_path = "/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2"
    blip_model_path = "/kaggle/input/clip-interrogator-models-x/model_large_caption.pth"
    ci_clip_model_name = "ViT-H-14/laion2b_s32b_b79k"
    clip_model_name = "ViT-H-14"
    clip_model_path = "/kaggle/input/clip-interrogator-models-x/CLIP-ViT-H-14-laion2B-s32B-b79K/open_clip_pytorch_model.bin"
    cache_path = "/kaggle/input/clip-interrogator-models-x"

### CLIP Interrogator uses OpenCLIP which supports many different pretrained CLIP models.
### For the best prompts for Stable Diffusion 1.X use ViT-L-14/openai for clip_model_name.
### For Stable Diffusion 2.0 use ViT-H-14/laion2b_s32b_b79k

In [ ]:
# replace tokenizer path to prevent downloading
blip_path = inspect.getfile(blip)

fin = open(blip_path, "rt")
data = fin.read()
data = data.replace(
    "BertTokenizer.from_pretrained('bert-base-uncased')", 
    "BertTokenizer.from_pretrained('/kaggle/input/clip-interrogator-models-x/bert-base-uncased')"
)
fin.close()

fin = open(blip_path, "wt")
fin.write(data)
fin.close()

# reload module
importlib.reload(blip)
del data

### This part I haven't really understand it

In [ ]:
model_config = clip_interrogator.Config(clip_model_name = CFG.ci_clip_model_name)
model_config.cache_path = CFG.cache_path

#blip_path = "/opt/conda/lib/python3.7/site-packages/blip/models/blip.py"
configs_path = os.path.join(os.path.dirname(os.path.dirname(blip_path)), 'configs')
med_config = os.path.join(configs_path, 'med_config.json')
blip_model = blip.blip_decoder(
    pretrained=CFG.blip_model_path,
    image_size=model_config.blip_image_eval_size, 
    vit=model_config.blip_model_type, 
    med_config=med_config
)

blip_model.eval()
blip_model = blip_model.to(model_config.device)
model_config.blip_model = blip_model


In [ ]:
clip_model = open_clip.create_model(CFG.clip_model_name, precision='fp16' if model_config.device == 'cuda' else 'fp32')
open_clip.load_checkpoint(clip_model, CFG.clip_model_path)
clip_model.to(model_config.device).eval()
model_config.clip_model = clip_model

clip_preprocess = open_clip.image_transform(
    clip_model.visual.image_size,
    is_train = False,
    mean = getattr(clip_model.visual, 'image_mean', None),
    std = getattr(clip_model.visual, 'image_std', None),
)

model_config.clip_preprocess = clip_preprocess

ci = clip_interrogator.Interrogator(model_config)

In [ ]:
# create the inverse transform
inverse_transform = transforms.Compose([
    transforms.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
    transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
    transforms.ToPILImage()
])

cos = torch.nn.CosineSimilarity(dim=1)

mediums_features_array = torch.stack([torch.from_numpy(t) for t in ci.mediums.embeds]).to(ci.device)
movements_features_array = torch.stack([torch.from_numpy(t) for t in ci.movements.embeds]).to(ci.device)
flavors_features_array = torch.stack([torch.from_numpy(t) for t in ci.flavors.embeds]).to(ci.device)

class MyClip:
    def __init__(self, model = ci, withFlave = True):
        self.model = model
        self.withFlave = withFlave
    def img_list(self, img_list):
        if len(img_list) == 1 and not isinstance(img_list, list) and not isinstance(img_list, torch.Tensor):
            img_list = [img_list]
        if isinstance(img_list[0], torch.Tensor):
            img_list = [inverse_transform(img) for img in img_list]
        else:
            pass
        return img_list
    def predict_prompt(self, img_list):
        img_list = self.img_list(img_list)
        prompts = []
        for img in img_list:
            prompt = self.model.generate_caption(img)
            if self.withFlave:
                medium, movement, flaves = self.feature(img)
                if prompt.startswith(medium):
                    prompt = f"{prompt}, {movement}, {flaves}"
                else:
                    prompt = f"{prompt}, {medium}, {movement}, {flaves}"
                prompt = clip_interrogator._truncate_to_fit(prompt, self.model.tokenize)
            prompts.append(prompt)
        return prompts[0] if len(prompts)==1 else prompts
    def feature(self, img):
        image_features = self.model.image_to_features(img)
        medium = [self.model.mediums.labels[i] for i in cos(image_features, mediums_features_array).topk(1).indices][0]
        movement = [self.model.movements.labels[i] for i in cos(image_features, movements_features_array).topk(1).indices][0]
        flaves = ", ".join([self.model.flavors.labels[i] for i in cos(image_features, flavors_features_array).topk(3).indices])
        return medium, movement, flaves
    
    def predict_emb(self, img_list):
        prompts = self.predict_prompt(img_list)
        embeddings = Prompt2Emb(prompts)
        return embeddings[0] if len(embeddings)==1 else embeddings

my_clip = MyClip()

### 1.1.2 Test sample images
### check scores and imaging for sample

In [ ]:
my_clip = MyClip(withFlave = True)
if not submission:
    sample_prompt_li = pd.read_csv("/kaggle/input/stable-diffusion-image-to-prompts/prompts.csv")
    sample_img_li = ['/kaggle/input/stable-diffusion-image-to-prompts/images/'+imgId+'.png' for imgId in sample_prompt_li['imgId']]
    sample_prompt_li = [prompt for prompt in sample_prompt_li['prompt']]
    sample_img_li = [Image.open(img).convert("RGB") for img in sample_img_li]

    prompt_clip_li = my_clip.predict_prompt(sample_img_li)
    test_li = st_model.encode(prompt_clip_li)

    #score_li=scoring(sample_prompt_li, prompt_clip_li)
    score_li=scoring(sample_prompt_li, test_li)
    print(score_li)
    print(np.average(score_li))

    image_grid(sample_img_li, sample_prompt_li, prompt_clip_li, score_li, 1, 7)

In [ ]:
my_clip = MyClip(withFlave = True)
if not submission:
    sample_prompt_li = pd.read_csv("/kaggle/input/stable-diffusion-image-to-prompts/prompts.csv")
    sample_img_li = ['/kaggle/input/stable-diffusion-image-to-prompts/images/'+imgId+'.png' for imgId in sample_prompt_li['imgId']]
    sample_prompt_li = [prompt for prompt in sample_prompt_li['prompt']]
    sample_img_li = [Image.open(img).convert("RGB") for img in sample_img_li]

    prompt_clip_li = my_clip.predict_prompt(sample_img_li)
    test_li = st_model.encode(prompt_clip_li)

    #score_li=scoring(sample_prompt_li, prompt_clip_li)
    score_li=scoring(sample_prompt_li, test_li)
    print(score_li)
    print(np.average(score_li))

## 1.2 ViT-B-16 Model

### 1.2.1 Build model¶

In [ ]:
class ViT_CFG:
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model_path = '/kaggle/input/saved-vit-model/vit_large_patch16_384_1_64_0.0001_0.6564.pth'
    model_name = 'vit_large_patch16_384'
    input_size = (384, 384)
    batch_size = 64

In [ ]:
import timm

vit_model = timm.create_model(
    model_name = ViT_CFG.model_name,
    pretrained = False,
    num_classes = 384
)
state_dict = torch.load(ViT_CFG.model_path)
vit_model.load_state_dict(state_dict)
vit_model.to(ViT_CFG.device)
vit_model.eval()

In [ ]:
import torch.nn.functional as F
class MyViT:
    def __init__(self, model = vit_model, tta=2):
        self.model = model
        self.tta = tta
        self.transformPIL = transforms.Compose([
                        transforms.Resize(ViT_CFG.input_size),
                        transforms.RandomHorizontalFlip(p=0.5),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                        ])
        
    def img_list(self, img_list): # The output img_list need to be torch.tensor
        if isinstance(img_list, torch.Tensor):
            if img_list.dim() == 3:
                img_list = img_list.unsqueeze(0)
            img_list = F.interpolate(img_list, size=ViT_CFG.input_size, mode='bilinear', align_corners=False).to(ViT_CFG.device)
        else:
            if isinstance(img_list, Image.Image):
                img_list = [img_list]
            img_list = [self.transformPIL(img).to(ViT_CFG.device) for img in img_list]
        return img_list
    def predict_emb(self, img_list):
        img_list = self.img_list(img_list)
        embeddings = []
        for img_emb in img_list:
            to_stack = np.empty((0, CFG.embedding_length))
            for t in np.arange(self.tta):
                embedding = self.model(img_emb.unsqueeze(0)).squeeze(0).detach().cpu().numpy()
                #embedding = embedding/(abs(embedding).max()+0.0000001)
                to_stack = np.vstack((to_stack,embedding))
            embedding = np.average(to_stack, axis=0)
            embedding = embedding / np.linalg.norm(embedding)
            embeddings.append(embedding)
        return embeddings
my_vit = MyViT()

### 1.2.2 Test sample images

In [ ]:
my_vit = MyViT(tta=2)
if not submission:
    predict_emb_li = my_vit.predict_emb(sample_img_li)
    #predict_emb_li = predict_emb_li /( np.abs(predict_emb_li).max(axis=-1, keepdims=True) + 0.0000001)
    #predict_emb_li = normalize(predict_emb_li) # I am not sure why
    score_li = scoring(sample_prompt_li, predict_emb_li)
    print(score_li)
    print(np.average(score_li))

## 1.3 OFA model

### https://www.kaggle.com/code/parikshitsharma2001/lb-0-55161-clipinterrogator-ofa-vit

### 1.3.1 Build Model

In [ ]:
class OFA_CFG:
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    #model_path = '/kaggle/input/stable-diffusion-vit-baseline-train/vit_base_patch16_224.pth'
    #model_path = '/kaggle/input/vit-large/vit_large_patch16_224.pth'
    #model_name = 'vit_large_patch16_224'
    #model_path = '/kaggle/input/timm-vit-huge-patch14-224-in21k/pytorch_model.bin'
    #model_name = 'vit_huge_patch14_224_in21k'
    #input_size = (224, 224)
    #batch_size = 64

In [ ]:
CKPT_DIR = "/kaggle/input/stable-diffusion-data/OFA-large-caption/"

ofa_tokenizer = OFATokenizer.from_pretrained(CKPT_DIR)
ofa_model = OFAModel.from_pretrained(CKPT_DIR, use_cache=False).cuda()
txt = " what does the image describe?"
ask_inputs = ofa_tokenizer([txt], return_tensors="pt").input_ids

In [ ]:
mean, std = [1, 1, 1], [1, 1, 1]
#mean, std = [0.0, 0.0, 0.0], [1.0, 1.0, 1.0]
resolution = 512 # I change from 480 (default) to the original resolution, or we just remove resize
ofa_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        #transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(), 
        transforms.Normalize(mean=mean, std=std)
#        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

In [ ]:
class MyOFA:
    def __init__(self, model = ofa_model):
        self.model = model
    def img_list(self, img_list):
        if len(img_list) == 1 and not isinstance(img_list, list) and not isinstance(img_list, torch.Tensor):
            img_list = [img_list]
        if isinstance(img_list[0], Image.Image):
            img_list = [ofa_transform(img).unsqueeze(0).to(OFA_CFG.device) for img in img_list]
        elif isinstance(img_list[0], torch.Tensor):
            img_list = [img.unsqueeze(0) for img in img_list]
        else:
            pass
        return img_list
        
    def predict_prompt(self, img_list):
        img_list = self.img_list(img_list)
        prompts = []
        for img in img_list:
            emb_ofa = self.model.generate(ask_inputs.cuda(), patch_images=img, num_beams=5, no_repeat_ngram_size=2)
            prompt = ofa_tokenizer.batch_decode(emb_ofa, skip_special_tokens=True)[0]
            prompts.append(prompt)
        return prompts[0] if len(prompts)==1 else prompts

    def predict_emb(self, img_list):
        prompts = self.predict_prompt(img_list)
        embeddings = Prompt2Emb(prompts)
        embeddings = embeddings / np.linalg.norm(embeddings)
        return embeddings[0] if len(embeddings)==1 else embeddings

my_ofa = MyOFA()

In [ ]:
#from gensim.parsing.preprocessing import remove_stopwords
my_ofa = MyOFA()
if not submission:
    sample_prompt_li = pd.read_csv("/kaggle/input/stable-diffusion-image-to-prompts/prompts.csv")
    sample_img_li = ['/kaggle/input/stable-diffusion-image-to-prompts/images/'+imgId+'.png' for imgId in sample_prompt_li['imgId']]
    sample_prompt_li = [prompt for prompt in sample_prompt_li['prompt']]
    sample_img_li = [Image.open(img).convert("RGB") for img in sample_img_li]

    prompt_ofa_li = my_ofa.predict_prompt(sample_img_li)
    #print(prompt_ofa_li)
    embedding_ofa_li = my_ofa.predict_emb(sample_img_li)
    
    score_li=scoring(sample_prompt_li, embedding_ofa_li)
    print(score_li)
    print(np.average(score_li))

    image_grid(sample_img_li, sample_prompt_li, prompt_ofa_li, score_li, 1, 7)

# 3. Online Data (need internet)

### ref to https://www.kaggle.com/code/debarshichanda/pytorch-blip-training/notebook

## 3.1 download dataset

### https://github.com/poloclub/diffusiondb/blob/main/notebooks/example-loading.ipynb

### check data randomly

# 4. Ensembling Model

## 4.1 Ensembling

In [ ]:
from torch import nn, optim
​
Init = torch.tensor([[0.52], [0.3], [0.18]]).repeat(1, 384)
​
class Ensemble_Model(nn.Module):
    def __init__(self, model_li):
        super().__init__()
        self.model1 = model_li[0]
        self.model2 = model_li[1]
        self.model3 = model_li[2]
        #        self.weights = torch.nn.Parameter(torch.tensor([0.52, 0.3, 0.18],dtype=torch.float32, requires_grad=True))
        self.weights = torch.nn.Parameter(torch.tensor(Init, dtype=torch.float32, requires_grad=True))
#        # Set weights to sum up to 1
    def forward(self, x):
        self.weights /= self.weights.sum(dim=0)
        # Calculate the embeddings for each model and compute the weighted sum
        embeddings = torch.zeros((x.size(0), self.model1.embedding_size))
        for i, model in enumerate([self.model1, self.model2, self.model3]):
            model_embedding = model.predict_emb(x)
            embeddings += model_embedding * self.weights[i]
#        embeddings /= torch.sum(self.weights)
        return embeddings

    def predict(self, X):
        embedding1 = self.model1.predict_emb(X)
        embedding2 = self.model2.predict_emb(X)
        embedding3 = self.model3.predict_emb(X)
        embeddings = torch.tensor(embedding1) * self.weights[0]
        + torch.tensor(embedding2) * self.weights[1]
        + torch.tensor(embedding3) * self.weights[2]
        return embeddings
    
    def parameters(self):
        return [self.weights]

## 4.2 Test ensembling model with sample images

In [ ]:
if not submission:
    model_li = [my_vit, my_clip, my_ofa]
    vit_weight_li = [0.6,0.7,0.8]
    for vit_weight in vit_weight_li:
        print('ViT weight is:', '%.3f'%vit_weight)
        weight_li = np.arange(0, 1-vit_weight+0.001, 0.05)
        for weight in weight_li:
            weights = [vit_weight, weight, 1-vit_weight-weight]
            MyModel = Ensemble_Model(model_li, weights = weights)
            embedding_li = MyModel.predict(sample_img_li)
            score_li = scoring(sample_prompt_li,embedding_li)
            print("clip w:",'%.3f'%weight, 'avg_score:','%.3f'%(np.average(score_li)),['%.3f'%score for score in score_li])

# 5. Submission¶

### This is a simpling decode the generated prompts to the vector with 384 dimensional, so 7x384 = 2688

In [ ]:
# 定義圖像資料集類別
class ImageDataset(Dataset):
    def __init__(self, comp_path, image_names):
        self.comp_path = comp_path
        self.image_names = image_names
        
        self.transform = transforms.Compose([
#            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, index):
        # 讀取圖像並將其轉換為 RGB 格式
        image_name = self.image_names[index]
        image_path = os.path.join(self.comp_path, 'images', image_name)
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)#.unsqueeze(0)#.to(ViT_CFG.device)
        return image

In [ ]:
comp_path = Path('/kaggle/input/stable-diffusion-image-to-prompts/')
image_names = os.listdir(comp_path / 'images')
imgIds = [i.split('.')[0] for i in image_names]

eIds = list(range(CFG.embedding_length))

imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, CFG.embedding_length),
        np.tile(range(CFG.embedding_length), len(imgIds))
    )
]

img_li = ImageDataset(comp_path, image_names)  # 創建圖像資料集對象
# 創建 DataLoader 對象
dataloader = DataLoader(dataset=img_li,  # 使用上面創建的圖像資料集對象
                        batch_size=2,  # 每批次讀取的圖像數量
#                        num_workers=1,  # 用於讀取數據的工作進程數
                        shuffle=False)  # 是否打亂資料集順序，此處不打亂

In [ ]:
df_submission = pd.read_csv(comp_path / 'sample_submission.csv', index_col='imgId_eId')
assert sorted(imgId_eId) == sorted(df_submission.index)

my_model = Ensemble_Model([my_vit, my_clip, my_ofa])
data_path = '/kaggle/input/ensembleweight-hsieh/'
if os.path.exists(data_path+'ensemble_model_weights.pth'):
    my_model.load_state_dict(torch.load(data_path+'ensemble_model_weights.pth'))
    accuracy_li = np.loadtxt(data_path+'/accuracy.txt')
    print('Using from last training point')
else:
    accuracy_li = np.array([])

predict_emb_li = []
for _, inputs in enumerate(tqdm(dataloader, total=len(dataloader))):
    inputs = inputs.to(CFG.device)
    
    prompt = MyModel.predict(inputs)
    predict_emb_li.extend(prompt)

predict_emb_li = np.array(predict_emb_li).flatten()

print('Size',np.shape(predict_emb_li))
submission = pd.DataFrame(
    index=imgId_eId,
    data=predict_emb_li,
    columns=['val']
).rename_axis('imgId_eId')

#display(df_submission)
#score_li = scoring(np.array(df_submission.sort_index()['val']).reshape(7,384),np.array(submission.sort_index()['val']).reshape(7, 384))
#print(score_li, np.average(score_li))
#display(submission)

submission.to_csv('submission.csv')